# OAM Component Contribution - Simple DynamoDB Example

This notebook demonstrates how to contribute a new OAM component using a simplified DynamoDB table as an example.

## What We'll Create

1. **Crossplane XRD** - Define the custom resource schema
2. **Crossplane Composition** - Define how the resource is composed
3. **KubeVela ComponentDefinition** - Make it usable in KubeVela applications
4. **Test Application** - Verify everything works

## Prerequisites

- Cluster set up from `00_Env-setup.ipynb`
- Crossplane installed with AWS provider
- KubeVela installed

## Step 1: Create an XRD for DynamoDB Table

This XRD defines a simple DynamoDB table with only the essential fields (Experience API):
- Table name
- Hash key (partition key)
- Attributes
- Region

You should also expose other fields but setup with sensible defaults and potentially hard code fields that are not modifiable in your platform (Opinionated Platform).

**Note:** We're using Crossplane XRD v2 API with `Cluster` scope for maintaining backward-compatibility with version v1. 

In [1]:
%%bash
# Create directory for Crossplane resources
mkdir -p crossplane/dynamodb

# Create simplified XRD using v2 API
cat > crossplane/dynamodb/xrd.yaml << 'EOF'
apiVersion: apiextensions.crossplane.io/v2
kind: CompositeResourceDefinition
metadata:
  name: xdynamodbtables.demo.kubecon.io
spec:
  scope: Cluster  # Cluster-scoped resources (v1 compatible behavior)
  group: demo.kubecon.io
  names:
    kind: XDynamoDBTable
    plural: xdynamodbtables
  versions:
    - name: v1alpha1
      served: true
      referenceable: true
      schema:
        openAPIV3Schema:
          type: object
          properties:
            spec:
              type: object
              properties:
                name:
                  type: string
                  description: "Name of the DynamoDB table"
                region:
                  type: string
                  description: "AWS region"
                hashKey:
                  type: string
                  description: "Hash (partition) key attribute name"
                attributes:
                  type: array
                  description: "Attribute definitions"
                  items:
                    type: object
                    properties:
                      name:
                        type: string
                      type:
                        type: string
                        enum: ["S", "N", "B"]
                tags:
                  type: object
                  additionalProperties:
                    type: string
                  description: "AWS resource tags"
              required:
                - name
                - region
                - hashKey
                - attributes
            status:
              type: object
              properties:
                tableArn:
                  type: string
                tableId:
                  type: string
EOF

echo "✓ XRD file created at crossplane/dynamodb/xrd.yaml (using v2 API with Cluster scope)"
echo ""

✓ XRD file created at crossplane/dynamodb/xrd.yaml (using v2 API with Cluster scope)



### Apply the XRD

In [2]:
%%bash

kubectl config use-context k3d-kubecon-NA25

kubectl apply -f crossplane/dynamodb/xrd.yaml

echo ""
echo "Waiting for XRD to be established..."
sleep 3

echo ""
echo "✓ Verifying XRD creation:"
kubectl get xrd xdynamodbtables.demo.kubecon.io


Switched to context "k3d-kubecon-NA25".
compositeresourcedefinition.apiextensions.crossplane.io/xdynamodbtables.demo.kubecon.io created

Waiting for XRD to be established...

✓ Verifying XRD creation:
NAME                              ESTABLISHED   OFFERED   AGE
xdynamodbtables.demo.kubecon.io   True                    4s


## Step 2: Create A Composition

The Composition defines how to create the actual DynamoDB table from our XRD.

In [3]:
%%bash
cat > crossplane/dynamodb/composition.yaml << 'EOF'
apiVersion: apiextensions.crossplane.io/v1
kind: Composition
metadata:
  name: dynamodb-table.demo.kubecon.io
spec:
  compositeTypeRef:
    apiVersion: demo.kubecon.io/v1alpha1
    kind: XDynamoDBTable
  
  mode: Pipeline
  pipeline:
    - step: create-table
      functionRef:
        name: function-patch-and-transform
      input:
        apiVersion: pt.fn.crossplane.io/v1beta1
        kind: Resources
        resources:
          - name: table
            base:
              apiVersion: dynamodb.aws.upbound.io/v1beta1
              kind: Table
              spec:
                forProvider:
                  billingMode: PAY_PER_REQUEST
            
            patches:
              # Patch table name with tenant-atlantis prefix
              - type: FromCompositeFieldPath
                fromFieldPath: spec.name
                toFieldPath: metadata.name
              
              # Patch region
              - type: FromCompositeFieldPath
                fromFieldPath: spec.region
                toFieldPath: spec.forProvider.region
              
              # Patch hash key
              - type: FromCompositeFieldPath
                fromFieldPath: spec.hashKey
                toFieldPath: spec.forProvider.hashKey
              
              # Patch attributes
              - type: FromCompositeFieldPath
                fromFieldPath: spec.attributes
                toFieldPath: spec.forProvider.attribute
              
              # Patch tags
              - type: FromCompositeFieldPath
                fromFieldPath: spec.tags
                toFieldPath: spec.forProvider.tags
              
              # Patch status back
              - type: ToCompositeFieldPath
                fromFieldPath: status.atProvider.arn
                toFieldPath: status.tableArn
              
              - type: ToCompositeFieldPath
                fromFieldPath: status.atProvider.id
                toFieldPath: status.tableId
EOF

echo "✓ Composition file created at crossplane/dynamodb/composition.yaml"
echo ""

✓ Composition file created at crossplane/dynamodb/composition.yaml



### Apply the Composition

In [4]:
%%bash
kubectl config use-context k3d-kubecon-NA25

echo "Checking Crossplane version..."
kubectl get provider.pkg.crossplane.io -A 2>/dev/null | head -5

echo ""
echo "Applying Composition..."
if kubectl apply -f crossplane/dynamodb/composition.yaml 2>&1 | tee /tmp/composition-apply.log; then
    echo ""
    echo "✓ Composition applied successfully"
else
    echo ""
    echo "❌ Composition apply failed. Checking error..."
    cat /tmp/composition-apply.log
    exit 1
fi

echo ""
echo "Waiting for Composition to be ready..."
for i in {1..10}; do
    if kubectl get composition dynamodb-table.demo.kubecon.io &>/dev/null; then
        echo "✓ Composition is ready"
        break
    fi
    echo "  Waiting... ($i/10)"
    sleep 2
done

echo ""
echo "✓ Verifying Composition creation:"
kubectl get composition dynamodb-table.demo.kubecon.io || echo "⚠ Composition not found - check the error above"

Switched to context "k3d-kubecon-NA25".
Checking Crossplane version...
NAME                            INSTALLED   HEALTHY   PACKAGE                                                 AGE
provider-kubernetes             True        True      xpkg.upbound.io/upbound/provider-kubernetes:v0.16.0     5m50s
upbound-provider-aws-dynamodb   True        True      xpkg.upbound.io/upbound/provider-aws-dynamodb:v1.23.2   6m33s
upbound-provider-aws-s3         True        True      xpkg.upbound.io/upbound/provider-aws-s3:v1.23.2         6m33s
upbound-provider-family-aws     True        True      xpkg.upbound.io/upbound/provider-family-aws:v1.23.2     6m13s

Applying Composition...
composition.apiextensions.crossplane.io/dynamodb-table.demo.kubecon.io created

✓ Composition applied successfully

Waiting for Composition to be ready...
✓ Composition is ready

✓ Verifying Composition creation:
NAME                             XR-KIND          XR-APIVERSION              AGE
dynamodb-table.demo.kubecon.io  

## Step 3: Create KubeVela ComponentDefinition (CUE)

This makes the DynamoDB table available as a KubeVela component.

In [5]:
%%bash
mkdir -p kubevela/components/dynamodb

cat > kubevela/components/dynamodb/dynamodb.cue << 'EOF'
"simple-dynamodb": {
    type: "component"
    attributes: {
        workload: definition: {
            apiVersion: "demo.kubecon.io/v1alpha1"
            kind:       "XDynamoDBTable"
        }
        status: {
            healthPolicy: #"""
                isHealth: bool | *false
                if context.output.status != _|_ {
                    if context.output.status.conditions != _|_ {
                        for c in context.output.status.conditions {
                            if c.type == "Ready" && c.status == "True" {
                                isHealth: true
                            }
                        }
                    }
                }
                """#
            customStatus: #"""
                message: string | *"Provisioning table..."
                if context.output.status != _|_ {
                    if context.output.status.tableArn != _|_ {
                        message: "Table ARN: " + context.output.status.tableArn
                    }
                }
                """#
        }
    }
}

template: {
    output: {
        apiVersion: "demo.kubecon.io/v1alpha1"
        kind:       "XDynamoDBTable"
        metadata: {
            name:      "tenant-atlantis-" + parameter.name
            namespace: context.namespace
        }
        spec: {
            name:       "tenant-atlantis-" + parameter.name
            region:     parameter.region
            hashKey:    parameter.hashKey
            attributes: parameter.attributes
            tags: {
                "gwcp:v1:dept":                            "000"
                "gwcp:v1:provisioned-resource:created-by": "kubecon-NA25"
                "gwcp:v1:quadrant:name":                   "dev"
                "gwcp:v1:resource-type:managed-by":        "pod-atlantis"
                "gwcp:v1:resource-type:managed-tool":      "crossplane"
                "gwcp:v1:star-system:name":                "kubecon"
                "gwcp:v1:tenant:name":                     "atlantis"
                "gwcp:v1:tenant:app-name":                 context.appName
            }
            crossplane: {
                compositionRef: {
                    name: "dynamodb-table.demo.kubecon.io"
                }
            }
        }
    }

    parameter: {
        // +usage=Name of the DynamoDB table (will be prefixed with tenant-atlantis-)
        name: string
        
        // +usage=AWS region
        region: string
        
        // +usage=Hash key attribute name
        hashKey: string
        
        // +usage=Attribute definitions
        attributes: [...{
            // +usage=Attribute name
            name: string
            // +usage=Attribute type (S=String, N=Number, B=Binary)
            type: "S" | "N" | "B"
        }]
    }
}
EOF

echo "✓ KubeVela ComponentDefinition CUE file created with expected prefix, tags, and Crossplane v2 compositionRef"
echo ""


✓ KubeVela ComponentDefinition CUE file created with expected prefix, tags, and Crossplane v2 compositionRef



### Apply the ComponentDefinition


In [6]:
%%bash
# Check if vela CLI is available
kubectl config use-context k3d-kubecon-NA25

if command -v vela &> /dev/null; then
    echo "Using vela CLI to apply ComponentDefinition..."
    cd kubevela/components/dynamodb
    vela def apply dynamodb.cue
    echo ""
    echo "✓ ComponentDefinition applied"
else
    echo "⚠ vela CLI not found. Please install it and try again."
    exit 1
fi

echo ""
echo "Checking for ComponentDefinition:"
kubectl get componentdefinition simple-dynamodb -n vela-system 2>/dev/null || echo "Not installed yet"

Switched to context "k3d-kubecon-NA25".
Using vela CLI to apply ComponentDefinition...
ComponentDefinition simple-dynamodb created in namespace vela-system.

✓ ComponentDefinition applied

Checking for ComponentDefinition:
NAME              WORKLOAD-KIND    DESCRIPTION
simple-dynamodb   XDynamoDBTable   


## Step 4: Test with Composite Resource (Direct XRD Test)

Let's test our XRD and Composition directly before creating the KubeVela application.

In [7]:
%%bash
mkdir -p test

cat > test/test-table.yaml << 'EOF'
apiVersion: demo.kubecon.io/v1alpha1
kind: XDynamoDBTable
metadata:
  name: tenant-atlantis-test-simple-table
  namespace: default
spec:
  name: tenant-atlantis-test-simple-table
  region: us-west-2
  hashKey: id
  attributes:
    - name: id
      type: "S"
  tags:
    "gwcp:v1:dept": "000"
    "gwcp:v1:provisioned-resource:created-by": "kubecon-NA25"
    "gwcp:v1:quadrant:name": "dev"
    "gwcp:v1:resource-type:managed-by": "pod-atlantis"
    "gwcp:v1:resource-type:managed-tool": "crossplane"
    "gwcp:v1:star-system:name": "kubecon"
    "gwcp:v1:tenant:name": "atlantis"
    "gwcp:v1:tenant:app-name": "test-app"
  crossplane:
    compositionRef:
      name: dynamodb-table.demo.kubecon.io
EOF

echo "✓ Test composite resource created with tenant-atlantis prefix"


✓ Test composite resource created with tenant-atlantis prefix


In [8]:
%%bash

kubectl config use-context k3d-kubecon-NA25

echo "Applying test composite resource..."
kubectl apply -f test/test-table.yaml

echo ""
echo "Waiting a few seconds for reconciliation..."
sleep 5

echo ""
echo "✓ Checking composite resource:"
kubectl get xdynamodbtables -n default

echo ""
echo "✓ Checking underlying DynamoDB Table resource:"
kubectl get table -n default 2>/dev/null || echo "Table resource not created yet (AWS provider may not be configured)"

Switched to context "k3d-kubecon-NA25".
Applying test composite resource...
xdynamodbtable.demo.kubecon.io/tenant-atlantis-test-simple-table created

Waiting a few seconds for reconciliation...

✓ Checking composite resource:
NAME                                SYNCED   READY   COMPOSITION                      AGE
tenant-atlantis-test-simple-table                    dynamodb-table.demo.kubecon.io   6s

✓ Checking underlying DynamoDB Table resource:


## Step 5: Create KubeVela ApplicationSSSSSS

If ComponentDefinition was successfully installed, we can create a KubeVela application.

In [9]:
%%bash
cat > test/app.yaml << 'EOF'
apiVersion: core.oam.dev/v1beta1
kind: Application
metadata:
  name: my-dynamodb-app
  namespace: default
spec:
  components:
    - name: users-table
      type: simple-dynamodb
      properties:
        name: users-table
        region: us-west-2
        hashKey: userId
        attributes:
          - name: userId
            type: "S"
EOF

echo "✓ KubeVela application manifest created"

✓ KubeVela application manifest created


In [10]:
%%bash
kubectl config use-context k3d-kubecon-NA25

# Only apply if ComponentDefinition exists
if kubectl get componentdefinition simple-dynamodb -n vela-system &>/dev/null; then
    echo "Applying KubeVela application..."
    vela up -f test/app.yaml
    
    echo ""
    sleep 3
    
    echo "✓ Application status:"
    vela status my-dynamodb-app
else
    echo "⚠ ComponentDefinition not found. Skipping application deployment."
    echo "To deploy: Install vela CLI and run the ComponentDefinition cell first."
fi

Switched to context "k3d-kubecon-NA25".
Applying KubeVela application...
Applying an application in vela K8s object format...
✅ App has been deployed 🚀🚀🚀
    Port forward: vela port-forward my-dynamodb-app
             SSH: vela exec my-dynamodb-app
         Logging: vela logs my-dynamodb-app
      App status: vela status my-dynamodb-app
        Endpoint: vela status my-dynamodb-app --endpoint
Application default/my-dynamodb-app applied.

✓ Application status:


About:

  Name:      	my-dynamodb-app              
  Namespace: 	default                      
  Created at:	2025-11-07 15:50:49 -0800 PST
  Healthy:   	❌                           
  Details:   	runningWorkflow              



Workflow:

  mode: DAG-DAG
  finished: false
  Suspend: false
  Terminated: false
  Steps
  - id: l1cyqwk698
    name: users-table
    type: apply-component
    phase: running 
    message: wait healthy

Services:

  - Name: users-table  
    Cluster: local
    Namespace: default
    Type: simple-dynamodb
    Health: ❌ 
      Message: Provisioning table...
    No trait applied



## Step 6: Verification Commands

Check the status of all created resources.

In [11]:
%%bash
echo "=== Resource Status ==="
echo ""
kubectl config use-context k3d-kubecon-NA25

echo "1. XRD:"
kubectl get xrd xdynamodbtables.demo.kubecon.io

echo ""
echo "2. Composition:"
kubectl get composition dynamodb-table.demo.kubecon.io

echo ""
echo "3. Composite Resources:"
kubectl get xdynamodbtables -A

echo ""
echo "4. DynamoDB Tables (Managed Resources):"
kubectl get table -A 2>/dev/null || echo "No tables found (AWS provider may not be configured)"

echo ""
echo "5. ComponentDefinition:"
kubectl get componentdefinition simple-dynamodb -n vela-system 2>/dev/null || echo "Not installed"

echo ""
echo "6. KubeVela Applications:"
vela ls -A 2>/dev/null || echo "No applications"
vela status my-dynamodb-app


=== Resource Status ===

Switched to context "k3d-kubecon-NA25".
1. XRD:
NAME                              ESTABLISHED   OFFERED   AGE
xdynamodbtables.demo.kubecon.io   True                    22s

2. Composition:
NAME                             XR-KIND          XR-APIVERSION              AGE
dynamodb-table.demo.kubecon.io   XDynamoDBTable   demo.kubecon.io/v1alpha1   17s

3. Composite Resources:
NAME                                SYNCED   READY   COMPOSITION                      AGE
tenant-atlantis-test-simple-table   True     False   dynamodb-table.demo.kubecon.io   14s
tenant-atlantis-users-table         True     False   dynamodb-table.demo.kubecon.io   7s

4. DynamoDB Tables (Managed Resources):
NAME                                SYNCED   READY   EXTERNAL-NAME                       AGE
tenant-atlantis-test-simple-table                    tenant-atlantis-test-simple-table   5s
tenant-atlantis-users-table                          tenant-atlantis-users-table         5s

5. Componen

About:

  Name:      	my-dynamodb-app              
  Namespace: 	default                      
  Created at:	2025-11-07 15:50:49 -0800 PST
  Healthy:   	❌                           
  Details:   	runningWorkflow              



Workflow:

  mode: DAG-DAG
  finished: false
  Suspend: false
  Terminated: false
  Steps
  - id: l1cyqwk698
    name: users-table
    type: apply-component
    phase: running 
    message: wait healthy

Services:

  - Name: users-table  
    Cluster: local
    Namespace: default
    Type: simple-dynamodb
    Health: ❌ 
      Message: Provisioning table...
    No trait applied



## Step 7: Publishing Guidelines

To publish this component to a library:

### Directory Structure
```
library/
├── crossplane/
│   └── dynamodb/
│       ├── xrd.yaml
│       └── composition.yaml
└── kubevela/
    └── components/
        └── dynamodb/
            ├── dynamodb.cue
            └── metadata.cue
```

### Metadata File
Create `metadata.cue` with component information:


In [12]:
%%bash
# Create metadata file
cat > kubevela/components/dynamodb/metadata.cue << 'EOF'
Name: "simple-dynamodb"
DepartmentCode: 000
MaintainedBy: "platform-team"
CreatedBy: "kubecon-NA25"
Stage: "alpha"
Dependencies: ["crossplane-aws-provider"]
Description: "Simple DynamoDB table component for KubeCon demo"
Version: "v0.1.0"
EOF

echo "✓ Metadata file created"

✓ Metadata file created


## Cleanup

To clean up all resources created in this demo, use the dedicated cleanup notebook:

📓 **`01-OAM-cleanup.ipynb`**

The cleanup notebook will:
- Delete KubeVela Applications
- Delete Composite Resources
- Wait for managed resources cleanup
- Delete ComponentDefinition
- Delete Composition
- Delete XRD
- Verify cleanup completion

## Summary

In this notebook, we demonstrated a simplified OAM component contribution workflow:

1. ✓ Created a simplified **Crossplane XRD** with minimal fields
2. ✓ Created a **Composition** to define resource creation
3. ✓ Created a **KubeVela ComponentDefinition** (CUE file)
4. ✓ Tested with a **Composite Resource**
5. ✓ Created a **KubeVela Application** example
6. ✓ Provided **publishing guidelines** and metadata

### Key Simplifications

This simplified version includes only:
- Table name
- Region
- Hash key
- Basic attributes
- Pay-per-request billing

The full production version would include:
- Range keys, TTL, GSI/LSI
- Encryption, backup settings
- Tags and governance metadata
- Deletion protection

### Next Steps

1. Extend the schema with additional DynamoDB features
2. Add validation and default values
3. Create comprehensive tests

### Cleanup

When you're done, run the **`01-OAM-cleanup.ipynb`** notebook to remove all resources.

## Summary

In this notebook, we demonstrated a simplified OAM component contribution workflow:

1. ✓ Created a simplified **Crossplane XRD** with minimal fields
2. ✓ Created a **Composition** to define resource creation
3. ✓ Created a **KubeVela ComponentDefinition** (CUE file)
4. ✓ Tested with a **Composite Resource**
5. ✓ Created a **KubeVela Application** example
6. ✓ Provided **publishing guidelines** and metadata

### Key Simplifications

This simplified version includes only:
- Table name
- Region
- Hash key
- Basic attributes
- Pay-per-request billing

The full production version would include:
- Range keys, TTL, GSI/LSI
- Encryption, backup settings
- Tags and governance metadata
- Deletion protection

### Next Steps

1. Extend the schema with additional DynamoDB features
2. Add validation and default values
3. Create comprehensive tests